##Questão 1 
Reconstruir o índice considerando o conjunto de dados que indicamos. Esses são os dados coletados por Bernardi e os estaremos usando para facilitar a correção da atividade. Se você já estiver usando esses dados não precisa reconstruir o índice;

In [0]:
import pandas as pd
import numpy as np
import nltk
import re
import collections
import bisect
nltk.download('punkt')
nltk.download('rslp')
nltk.download('stopwords')


nltk.download('stopwords')
data = pd.read_csv('./results.csv', sep=',')


textData = pd.DataFrame(data['text'], columns=['text'])
textData['tokenizedText'] = data.apply(lambda row: nltk.word_tokenize(row['text'].lower(), language='portuguese'), axis=1)

stopwords = nltk.corpus.stopwords.words('portuguese')

index = {}
document = 0

for wordList in textData.tokenizedText:
  document += 1
  for word in wordList:
    if word not in stopwords and len(word) >= 3:      
      if word not in index.keys():
        index[word] = []
      index[word].append(document)
      
for elem in index.items():
  d = dict(collections.Counter(elem[1]))
  index[elem[0]] = list(d.items())
  
  
index_df = pd.DataFrame()
index_df['Word'] = index.keys()
index_df['Documents and Frequencies'] = index.values()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#Questão 2
Refinar o índice invertido de forma a também incluir o IDF (inverse document frequency) de cada termo do dicionário;

In [0]:
M = data.text.count()
for word in index:
  k = len(index[word])
  IDF = round(np.log((M+1)/k),2)
  index[word].append(IDF)
  
index_df['IDF'] = [i[-1] for i in index.values()]

index_df.head(10)

,Word,Documents and Frequencies,IDF
0,juíza,"[(1, 2), (2, 1), 4.83]",4.83
1,federal,"[(1, 2), (2, 2), (3, 1), (6, 1), (7, 3), (15, ...",1.63
2,ivani,"[(1, 1), (2, 1), 4.83]",4.83
3,silva,"[(1, 3), (2, 1), (6, 1), (14, 1), (26, 1), (73...",3.04
4,luz,"[(1, 3), (2, 1), (9, 1), (17, 1), (32, 2), (78...",2.63
5,brasília,"[(1, 1), (8, 1), (33, 1), (35, 1), (44, 1), (4...",2.23
6,proibiu,"[(1, 1), (2, 1), (119, 1), (162, 1), 4.14]",4.14
7,caráter,"[(1, 1), (15, 1), (36, 1), (60, 1), (89, 1), (...",3.22
8,liminar,"[(1, 1), (2, 3), (119, 1), (217, 1), 4.14]",4.14
9,nesta,"[(1, 2), (4, 1), (8, 1), (21, 1), (22, 2), (23...",0.93


#Questão 3

Implementar as seguintes versões do modelo vetorial:



*   Representação binária
*   TF
*   TF-IDF
*   BM25


As funções que criam esses modelos serão implementadas abaixo.



In [0]:
def binary(query, document):
  score = 0
  query_tokens = query.split()
  doc_tokens = document.split()
  
  for token in query_tokens:
    score += (token in doc_tokens)
    
  return score

In [0]:
def tf(query, document):
  score = 0
  doc_tokens = document.split()
  query_tokens = query.split()
  
  for word in query_tokens:
    score += doc_tokens.count(word)
  
  return score

In [0]:
def tf_idf(query, document):
  score = 0
  doc_tokens = document.split()
  query_tokens = query.split()
  
  for word in query_tokens:
    cwd = doc_tokens.count(word)
    if word in index:
      score += cwd * index[word][-1]
  
  return round(score,2)

In [0]:
def bm25(query, document, k):
  score = 0
  doc_tokens = document.split()
  query_tokens = query.split()
  
  words = [word for word in query_tokens if word in doc_tokens]
    
  for word in words:
    cwd = doc_tokens.count(word)
    dfw = 0
    if word in index:
      dfw = len(index[word][:-1])
    score += (((k+1) * cwd) / (cwd + k)) * np.log10(((M+1) / dfw)) if dfw != 0 else 0
  
  return round(score,2)

#Questão 4
Execute os algoritmos separadamente em 3 consultas de sua escolha e retorne os top-5 documentos mais similares à cada consulta;

In [0]:
queries = ['lula', 'netflix', 'educação']

def create_top5_models(query):
  n = 0
  db = []
  dtf = []
  dtfidf = []
  dbm25 = []
  for doc in data.text:
    doc = doc.lower()
    n += 1
    bisect.insort(db, (binary(query, doc), n))
    bisect.insort(dtf, (tf(query,doc), n))
    bisect.insort(dtfidf, (tf_idf(query,doc), n))
    bisect.insort(dbm25, (bm25(query,doc,20), n))
  
  db.reverse()
  dtf.reverse()
  dtfidf.reverse()
  dbm25.reverse()
  
  return db[:5], dtf[:5], dtfidf[:5], dbm25[:5]

In [0]:
top5_binary = ['','','']
top5_tf = ['','','']
top5_tf_idf = ['','','']
top5_bm25 = ['','','']

top5_binary[0], top5_tf[0], top5_tf_idf[0], top5_bm25[0] = create_top5_models(queries[0])
top5_binary[1], top5_tf[1], top5_tf_idf[1], top5_bm25[1] = create_top5_models(queries[1])
top5_binary[2], top5_tf[2], top5_tf_idf[2], top5_bm25[2] = create_top5_models(queries[2])


query_df = pd.DataFrame()

query_df['Query'] = queries
query_df['Binary'] = top5_binary
query_df['TF'] = top5_tf
query_df['TF-IDF'] = top5_tf_idf
query_df['BM25'] = top5_bm25

query_df.index+=1
query_df

,Query,Binary,TF,TF-IDF,BM25
1,lula,"[(1, 236), (1, 234), (1, 226), (1, 216), (1, 2...","[(9, 15), (3, 234), (2, 226), (2, 204), (1, 236)]","[(27.36, 15), (9.12, 234), (6.08, 226), (6.08,...","[(8.59, 15), (3.61, 234), (2.52, 226), (2.52, ..."
2,netflix,"[(1, 200), (0, 249), (0, 248), (0, 247), (0, 2...","[(1, 200), (0, 249), (0, 248), (0, 247), (0, 2...","[(4.83, 200), (0.0, 249), (0.0, 248), (0.0, 24...","[(2.1, 200), (0, 249), (0, 248), (0, 247), (0,..."
3,educação,"[(1, 248), (1, 240), (1, 239), (1, 233), (1, 2...","[(15, 221), (8, 222), (6, 239), (6, 130), (5, ...","[(33.45, 221), (17.84, 222), (13.38, 239), (13...","[(8.7, 221), (5.8, 222), (4.68, 239), (4.68, 1..."


#Questão 5

Compare os resultados encontrados e responda:


*   Quais modelos você acha que trouxe os melhores resultados? Por que? Inspecione os documentos retornados para melhor embasar sua resposta.

*   Calcule e reporte o overlap par-a-par entre os resultados de cada modelo, usando o índice de Jaccard.



##Quais modelos você acha que trouxe os melhores resultados? Por que? Inspecione os documentos retornados para melhor embasar sua resposta.

In [0]:
def get_top1_score(top5):
  return [top[0][0] for top in top5]

def get_top1_doc(top5):
  return [top[0][1] for top in top5]

def get_doc_title(docs):
  return [data.title[doc] for doc in docs]

In [0]:
score_top5_binary = get_top1_score(top5_binary)
score_top5_tf = get_top1_score(top5_tf)
score_top5_tf_idf = get_top1_score(top5_tf_idf)
score_top5_bm25 = get_top1_score(top5_bm25)

doc_top5_binary = get_top1_doc(top5_binary)
doc_top5_tf = get_top1_doc(top5_tf)
doc_top5_tf_idf = get_top1_doc(top5_tf_idf)
doc_top5_bm25 = get_top1_doc(top5_bm25)

titles_binary = get_doc_title(doc_top5_binary)
titles_tf = get_doc_title(doc_top5_tf)
titles_tf_idf = get_doc_title(doc_top5_tf_idf)
titles_bm25 = get_doc_title(doc_top5_bm25)

In [0]:
binary_table = pd.DataFrame()
binary_table['Query'] = queries
binary_table['Document'] = doc_top5_binary
binary_table['Document Title'] = titles_binary
binary_table['Binary Score'] = score_top5_binary

binary_table.index += 1
binary_table

,Query,Document,Document Title,Binary Score
1,lula,236,Bolsonaro anuncia 13º salário para o Bolsa Fam...,1
2,netflix,200,O ‘Abaporu’ visita São Paulo,1
3,educação,248,As três espanholas do Estado Islâmico: “Só que...,1


In [0]:
tf_table = pd.DataFrame()
tf_table['Query'] = queries
tf_table['Document'] = doc_top5_tf
tf_table['Document Title'] = titles_tf
tf_table['TF Score'] = score_top5_tf

tf_table.index += 1
tf_table

,Query,Document,Document Title,TF Score
1,lula,15,A tragédia da Ucrânia,9
2,netflix,200,O ‘Abaporu’ visita São Paulo,1
3,educação,221,A jovem deputada que jogou contra as cordas o ...,15


In [0]:
tf_idf_table = pd.DataFrame()
tf_idf_table['Query'] = queries
tf_idf_table['Document'] = doc_top5_tf_idf
tf_idf_table['Document Title'] = titles_tf_idf
tf_idf_table['TFIDF Score'] = score_top5_tf_idf

tf_idf_table.index += 1
tf_idf_table

,Query,Document,Document Title,TFIDF Score
1,lula,15,A tragédia da Ucrânia,27.36
2,netflix,200,O ‘Abaporu’ visita São Paulo,4.83
3,educação,221,A jovem deputada que jogou contra as cordas o ...,33.45


In [0]:
bm25_table = pd.DataFrame()
bm25_table['Query'] = queries
bm25_table['Document'] = doc_top5_bm25
bm25_table['Document Title'] = titles_bm25
bm25_table['BM25 Score'] = score_top5_bm25

bm25_table.index += 1
bm25_table

,Query,Document,Document Title,BM25 Score
1,lula,15,A tragédia da Ucrânia,8.59
2,netflix,200,O ‘Abaporu’ visita São Paulo,2.10
3,educação,221,A jovem deputada que jogou contra as cordas o ...,8.70


De acordo com as tabelas mostradas acima, podemos concluir que o algoritmo TF-IDF é o que apresenta os melhores scores para os documentos encontrados em cada query.

##Calcule e reporte o overlap par-a-par entre os resultados de cada modelo, usando o índice de Jaccard.

In [0]:
def inter_size(a,b):
  return len([elem for elem in a if elem in b])

def jaccard_index(a,b):
  n_a = len(a)
  n_b = len(b)
  n_ab = inter_size(a,b)
  exp = n_a + n_b - n_ab
  jaccard = n_ab / exp if exp != 0 else 0
  return jaccard

In [0]:
all_docs = []
all_docs.append(doc_top5_binary)
all_docs.append(doc_top5_bm25)
all_docs.append(doc_top5_tf)
all_docs.append(doc_top5_tf_idf)

matrix = [['Measure', 'Binary', 'TF', 'TF-IDF', 'BM25']]
titles = matrix
for i in range(len(all_docs)):
  line = []
  for j in range(len(all_docs)):
    jaccard = jaccard_index(all_docs[i], all_docs[j])
    line.append(round(jaccard,2))
  matrix.append(line)
for i in range(1, len(matrix)):
  matrix[i].insert(0, matrix[0][i])
  
pd.DataFrame(matrix)

,0,1,2,3,4
0,Measure,Binary,TF,TF-IDF,BM25
1,Binary,1,0.2,0.2,0.2
2,TF,0.2,1,1,1
3,TF-IDF,0.2,1,1,1
4,BM25,0.2,1,1,1


Atráves do indíce de Jaccard mostrado acima, comparando os algoritmos utilizados, podemos concluir que o modelo binário para essas consultas é o único que não apresenta, pelo indíce, igualdade com os outros modelos. 